# REMDのOpenMM実装
- ChatGPTを使用
- 一定時間ごとに、レプリカ間の座標を交換する(温度はそのまま)
- 速度は交換しない
- 交換確率が0.25になるよう温度を調整するための予備MD

In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
import numpy as np
from tqdm.notebook import tqdm
import random
import os
from datetime import datetime
from math import ceil

In [2]:
# 現在の日付と時刻を取得
current_time = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
# ディレクトリ名を作成
output_dir = f"./output/output_{current_time}"
# ディレクトリを作成
os.makedirs(output_dir, exist_ok=True)
print(f"ディレクトリ '{output_dir}' を作成しました！")

ディレクトリ './output/output_2024-12-09_18-07-20' を作成しました！


In [3]:
# パラメータ設定
params = {}
params['n_replicas'] = 8  # レプリカ数
replicas = [str(i) for i in np.arange(params['n_replicas'])]
print('Replicas:', replicas)
params['temperature_min'] = 300 # K
params['temperature_max'] = 500 # K
params['temperatures'] = np.linspace(params['temperature_min'], params['temperature_max'], params['n_replicas'])  # 温度 (Kelvin)

params['temperatures'] = params['temperatures'].tolist()
params['old_temperatures'] = copy.deepcopy(params['temperatures'])

print('Temperatures (K):', params['temperatures'])
params['acceptance_ratio'] = 0.25

params['dt'] = 0.002 # タイムステップ(ps)
params['n_steps'] = 2 * int(ceil(1e3 / params['dt'])) # シミュレーション総ステップ数(1us) 1e6
params['n_steps_exchange'] = 2 * int(ceil(1e0 / params['dt'])) # 交換を試みる間隔(1ns) 1e3
params['n_steps_save'] = 1 * int(ceil(1e0 / params['dt'])) # 保存間隔(100ps) 100
params['n_steps_equil'] = 50 * int(ceil(1e0 / params['dt'])) # 平衡化(10ns) 20000
params['adjust_interval'] = 40
params['n_steps_adjust'] = params['adjust_interval'] * params['n_steps_exchange'] # 10回交換後に温度調整]
params['learning_rate'] = 0.1 # 0~1

params['n_times_exchange'] = params['n_steps'] // params['n_steps_exchange']

params['nonbondedCutoff'] = 1.0 # nm
params['friction'] = 1.0 # /ps
params['restraint_force'] = 10 # kcal/mol/A^2

params['pdb_path'] = './structures/ala2_solvated.pdb'
params['ff'] = ['amber99sbildn.xml', 'tip3p.xml']

for key in params.keys():
    print(f'{key}: {params[key]}')

Replicas: ['0', '1', '2', '3', '4', '5', '6', '7']
Temperatures (K): [300.0, 328.57142857142856, 357.14285714285717, 385.7142857142857, 414.2857142857143, 442.8571428571429, 471.42857142857144, 500.0]
n_replicas: 8
temperature_min: 300
temperature_max: 500
temperatures: [300.0, 328.57142857142856, 357.14285714285717, 385.7142857142857, 414.2857142857143, 442.8571428571429, 471.42857142857144, 500.0]
old_temperatures: [300.0, 328.57142857142856, 357.14285714285717, 385.7142857142857, 414.2857142857143, 442.8571428571429, 471.42857142857144, 500.0]
acceptance_ratio: 0.25
dt: 0.002
n_steps: 1000000
n_steps_exchange: 1000
n_steps_save: 500
n_steps_equil: 25000
adjust_interval: 40
n_steps_adjust: 40000
learning_rate: 0.1
n_times_exchange: 1000
nonbondedCutoff: 1.0
friction: 1.0
restraint_force: 10
pdb_path: ./structures/ala2_solvated.pdb
ff: ['amber99sbildn.xml', 'tip3p.xml']


### 初期パラメータをJSONに保存

In [4]:
# パラメータをJSONに保存
import json
filepath = f"{output_dir}/params.json"
with open(filepath, mode="wt", encoding="utf-8") as f:
	json.dump(params, f, ensure_ascii=False, indent=2)

### CSVファイルのセットアップ

In [5]:
# 各レプリカサンプルの現在位置を保存するファイル
with open(f'{output_dir}/replicas.csv', 'a') as f:
    f.write('step,'+','.join(replicas)+'\n')

# レプリカ交換のaccept/rejectを保存するファイル
with open(f'{output_dir}/acceptance.csv', 'a') as f:
    f.write(f"step,replica1,replica2,acceptance\n")

# 温度更新ごとに、温度を保存するファイル
with open(f'{output_dir}/adjusted_temperatures.csv', 'a') as f:
    f.write('step,'+','.join(replicas)+'\n')

# 温度更新ごとに、直近の交換確率を保存するファイル
with open(f'{output_dir}/acceptance_rates.csv', 'a') as f:
    f.write('step,'+','.join([f'{i}_{i+1}' for i in range(params['n_replicas']-1)])+'\n')

## レプリカ交換関数の定義

In [6]:
kB = MOLAR_GAS_CONSTANT_R.value_in_unit(kilojoule_per_mole/kelvin)
print(kB)

0.00831446261815324


### レプリカ交換

In [7]:
# エネルギー交換関数
# 全部value_in_unitする
def attempt_exchange(replica1, replica2, step):
    E1 = simulations[replica1].context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(kilojoule_per_mole)
    E2 = simulations[replica2].context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(kilojoule_per_mole)
    kB = MOLAR_GAS_CONSTANT_R.value_in_unit(kilojoule_per_mole/kelvin)
    beta1 = 1 / (kB * params['temperatures'][replica1])
    beta2 = 1 / (kB * params['temperatures'][replica2])
    delta = (beta2 - beta1) * (E1 - E2)
    # print(delta)
    
    if delta < 0 or random.uniform(0, 1) < np.exp(-delta):
        # 交換を行う
        # print(f"Exchange accepted between replica {replica1} and {replica2}")
        # temp1 = temperatures[replica1]
        # temperatures[replica1] = temperatures[replica2]
        # temperatures[replica2] = temp1
        rep1 = replicas[replica1]
        replicas[replica1] = replicas[replica2]
        replicas[replica2] = rep1        
        positions1 = simulations[replica1].context.getState(getPositions=True).getPositions()
        positions2 = simulations[replica2].context.getState(getPositions=True).getPositions()
        simulations[replica1].context.setPositions(positions2)
        simulations[replica2].context.setPositions(positions1)
        # Write log
        with open(f'{output_dir}/acceptance.csv', 'a') as f:
            # f.write(','.join(replicas)+'\n')
            f.write(f"{step},{replica1},{replica2},1\n")
        return True
    else:
        # print(f"Exchange rejected between replica {replica1} and {replica2}")
        # Write log
        with open(f'{output_dir}/acceptance.csv', 'a') as f:
            # f.write(','.join(replicas)+'\n')
            f.write(f"{step},{replica1},{replica2},0\n")
        return False

### 温度調整

In [8]:
def adjust_temperatures(exchange_attempts, exchange_success, step):
    # temperatures: 現在の温度リスト
    # exchange_attempts[i], exchange_success[i]: レプリカ i と i+1 の交換試行数と成功数
    # target: 目標成功率(0.25)
    
    # 現在の温度の逆数 (beta) リストを用意 (等間隔調整などの参考)
    betas = [1.0/(MOLAR_GAS_CONSTANT_R.value_in_unit(kilojoule_per_mole/kelvin)*T) for T in params['temperatures']]
    
    # 各ペアの交換成功率を計算
    acceptance = []
    for i in range(len(params['temperatures'])-1):
        if exchange_attempts[i] > 0:
            acc = exchange_success[i]/exchange_attempts[i]
        else:
            acc = 0.0
        acceptance.append(acc)
    
    # 隣接温度間の調整
    # 非常に単純な方針：成功率が高いペア -> 温度差を小さく
    # 成功率が低いペア -> 温度差を大きく
    # 以下はbetasを微調整し、それに基づいてtemperaturesを更新する例
    #
    # delta_beta[i] = betas[i+1]-betas[i] がペアiの温度差を支配
    # 目標交換率からのズレによって delta_beta をスケール

    for i, acc in enumerate(acceptance):
        # 誤差
        diff = acc - params['acceptance_ratio']
        # diff > 0 (成功率が高い)：温度差を小さくしたい => beta差を小さくする
        # diff < 0 (成功率が低い)：温度差を大きくしたい => beta差を大きくする
    
        adjust_factor = 1.0 + params['learning_rate']*diff  # acc=0.25なら1.0に近い値
        # betas[i+1]とbetas[i]の中点を固定して差分を調整する簡易法
        # mid_beta = 0.5*(betas[i] + betas[i+1])
        # half_diff = 0.5*(betas[i+1]-betas[i])*adjust_factor
        beta_diff = (betas[i+1]-betas[i])*adjust_factor
        
        # betas[i] = mid_beta - half_diff
        # betas[i+1] = mid_beta + half_diff
        betas[i+1] = betas[i] + beta_diff
    
    # betasから温度を再計算
    new_temperatures = []
    for b in betas:
        new_T = 1.0/(b*MOLAR_GAS_CONSTANT_R.value_in_unit(kilojoule_per_mole/kelvin))
        new_temperatures.append(new_T)
    
    params['temperatures'] = new_temperatures # 新しい温度に更新
    
    print("Iteration:", step//params['n_steps_exchange'])
    print("Adjusted temperatures:", [T for T in params['temperatures']])
    print("Current acceptance rates:", [exchange_success[j]/exchange_attempts[j] if exchange_attempts[j] > 0 else 0 for j in range(params['n_replicas']-1)])

    with open(f'{output_dir}/adjusted_temperatures.csv', 'a') as f:
        f.write(str(step)+','+','.join([str(T) for T in params['temperatures']])+'\n')

    with open(f'{output_dir}/acceptance_rates.csv', 'a') as f:
        f.write(str(step)+','+','.join([str(exchange_success[j]/exchange_attempts[j]) if exchange_attempts[j] > 0 else '0' for j in range(params['n_replicas']-1)])+'\n')


## システムの作成

In [9]:
pdb = app.PDBFile(params['pdb_path'])
forcefield = app.ForceField(*params['ff'])

In [10]:
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=params['nonbondedCutoff']*nanometer,
    constraints=app.HBonds,
)
print('System created...')

# レプリカごとに異なるシミュレーションをセットアップ
integrators = []
simulations = []
for i, temp in enumerate(params['temperatures']):
    integrator = LangevinIntegrator(
        temp*kelvin,       # 温度
        params['friction']/picosecond,    # 摩擦係数
        params['dt']*picoseconds  # タイムステップ
    )
    integrators.append(integrator)
    simulation = app.Simulation(pdb.topology, system, integrator)
    # simulation.context.setPositions(pdb.positions)
    # simulation.context.setVelocitiesToTemperature(temp*kelvin)
    simulations.append(simulation)

System created...


## 平衡化

In [11]:
%%time
system_equil = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=params['nonbondedCutoff']*nanometer,
    constraints=app.HBonds,
)
# Positional restraints for all heavy-atoms for equilibration
pos_res = CustomExternalForce("k*periodicdistance(x, y, z, x0, y0, z0)^2;")
pos_res.addPerParticleParameter("k")
pos_res.addPerParticleParameter("x0")
pos_res.addPerParticleParameter("y0")
pos_res.addPerParticleParameter("z0")

for ai, atom in enumerate(pdb.topology.atoms()):
    if atom.name == 'CA':
        x = pdb.positions[ai][0].value_in_unit(nanometers)
        y = pdb.positions[ai][1].value_in_unit(nanometers)
        z = pdb.positions[ai][2].value_in_unit(nanometers)
        pos_res.addParticle(ai, [params['restraint_force']*kilocalories_per_mole/(angstrom**2), x, y, z])

system_equil.addForce(pos_res)
print('System_equil created...')

integrators_equil = []
simulations_equil = []
for i, temp in enumerate(params['temperatures']):
    integrator = LangevinIntegrator(
        temp*kelvin,       # 温度
        params['friction'],    # 摩擦係数
        params['dt']*picoseconds  # タイムステップ
    )
    # integrators_equil.append(integrator)
    simulation_equil = app.Simulation(pdb.topology, system_equil, integrator)
    simulation_equil.context.setPositions(pdb.positions)
    simulation_equil.minimizeEnergy()
    simulation_equil.context.setVelocitiesToTemperature(temp*kelvin)
    print("Equilibrating for", params['n_steps_equil'], "steps")
    simulation_equil.step(params['n_steps_equil'])
    # simulations_equil.append(simulation)
    simulations[i].context.setPositions(
        simulation_equil.context.getState(getPositions=True ).getPositions()
    )
    simulations[i].context.setVelocities(
        simulation_equil.context.getState(getVelocities=True).getVelocities()
    )

System_equil created...
Equilibrating for 25000 steps
Equilibrating for 25000 steps
Equilibrating for 25000 steps
Equilibrating for 25000 steps
Equilibrating for 25000 steps
Equilibrating for 25000 steps
Equilibrating for 25000 steps
Equilibrating for 25000 steps
CPU times: user 23.1 s, sys: 2.9 s, total: 26 s
Wall time: 26.5 s


## レポーターの追加

In [12]:
# append reporters
print('Adding Reporters...')
dcd_reporters = [app.DCDReporter(file=f'{output_dir}/replica_{i}.dcd', reportInterval=params['n_steps_save']) 
                 for i, temp in enumerate(params['temperatures'])]
state_data_reporters = [app.StateDataReporter(file=f'{output_dir}/replica_{i}.log', 
                                              reportInterval=params['n_steps_save'], 
                                              totalSteps=params['n_steps'], 
                                              step=True,
                                              potentialEnergy=True,
                                              temperature=True,
                                              speed=True, 
                                              progress=True, 
                                              elapsedTime=True) 
                        for i, temp in enumerate(params['temperatures'])]

# 各レプリカにreporterを追加
for i, temp in enumerate(params['temperatures']):
    simulations[i].reporters.append(dcd_reporters[i])
    simulations[i].reporters.append(state_data_reporters[i])

Adding Reporters...


## Production
- 隣接ペアの組み合わせを1回毎に変えるようにしたい
- accept/rejectのログを記録したい

In [13]:
for i in range(0, 8, 2):
    print(i, i+1)
for i in range(1, 8-1, 2):
    print(i, i+1)

0 1
2 3
4 5
6 7
1 2
3 4
5 6


In [14]:
# %%time
# シミュレーションループ
print('Production...')
is_even_step = True # 交換ペアを交互に変えるための変数
exchange_attempts = np.zeros(params['n_replicas']-1)
exchange_success = np.zeros(params['n_replicas']-1)

for step in tqdm(range(0, params['n_steps'], params['n_steps_exchange']), leave=False):
    # print(f'*** Step {step} ***"')
    for sim in simulations:
        sim.step(params['n_steps_exchange'])  # 各レプリカで実行

    # レプリカ間での交換試行
    if is_even_step:
        for i in range(0, params['n_replicas'], 2):
            success = attempt_exchange(i, i + 1, step)
            exchange_attempts[i] += 1
            if success:
                exchange_success[i] += 1
    else:
        for i in range(1, params['n_replicas']-1, 2):
            success = attempt_exchange(i, i + 1, step)
            exchange_attempts[i] += 1
            if success:
                exchange_success[i] += 1

    # n_steps_adjust回温度交換したら、交換確率に基づいて温度を更新
    if step % params['n_steps_adjust'] == 0 and step > 0:
        adjust_temperatures(exchange_attempts, exchange_success, step)
        # integratorの温度をセット
        for i, integrator in enumerate(integrators):
            integrator.setTemperature(params['temperatures'][i])

        # 交換確率をリセット
        for i in range(params['n_replicas'] - 1):
            exchange_attempts[i] = 0
            exchange_success[i] = 0
            
    is_even_step = not is_even_step
        
    with open(f'{output_dir}/replicas.csv', 'a') as f:
            f.write(str(step)+','+','.join(replicas)+'\n')

Production...


  0%|          | 0/1000 [00:00<?, ?it/s]

Iteration: 40
Adjusted temperatures: [300.0, 327.9393459318773, 356.3495201024536, 385.07209792018534, 414.6002520768095, 442.3905214159597, 471.2739018044751, 500.5448264716059]
Current acceptance rates: [0.047619047619047616, 0.0, 0.047619047619047616, 0.35, 0.09523809523809523, 0.2, 0.42857142857142855]
Iteration: 80
Adjusted temperatures: [300.0, 327.1775852871694, 355.55696240620046, 384.43385322390486, 413.7885058554698, 442.237678975963, 470.65586001036775, 499.90989163898706]
Current acceptance rates: [0.0, 0.0, 0.05, 0.0, 0.2, 0.05, 0.05]
Iteration: 120
Adjusted temperatures: [300.0, 326.4382682973654, 354.7658237358239, 383.7919452391537, 413.4653479953912, 441.9301475096361, 470.19741349586366, 499.43648955231765]
Current acceptance rates: [0.0, 0.0, 0.05, 0.15, 0.15, 0.1, 0.1]
Iteration: 160
Adjusted temperatures: [300.0, 326.00731485451405, 353.9851598595995, 383.1466994641984, 413.62900166043886, 441.3262228452784, 470.19741349586354, 498.9710642072105]
Current acceptance

In [15]:
# # 参考コード
# import math
# import random
# from simtk.openmm import app, Platform
# import simtk.openmm as mm
# import simtk.unit as unit

# # -------------------------
# # 準備段階: SystemとIntegratorなどの基本設定
# # -------------------------
# pdb = app.PDBFile('input.pdb')
# forcefield = app.ForceField('amber14-all.xml', 'amber14-tip3p.xml')

# system = forcefield.createSystem(
#     pdb.topology,
#     nonbondedMethod=app.PME,
#     nonbondedCutoff=1.0*unit.nanometer,
#     constraints=app.HBonds
# )

# # 初期的な温度リスト（粗い間隔から開始）
# T_min = 300.0*unit.kelvin
# T_max = 400.0*unit.kelvin
# n_replicas = 8
# temperatures = [T_min + i*(T_max - T_min)/(n_replicas-1) for i in range(n_replicas)]

# def create_simulations(temps):
#     simulations = []
#     for T in temps:
#         integrator = mm.LangevinIntegrator(T, 1.0/unit.picosecond, 0.002*unit.picoseconds)
#         sim = app.Simulation(pdb.topology, system, integrator)
#         sim.context.setPositions(pdb.positions)
#         sim.minimizeEnergy()
#         sim.context.setVelocitiesToTemperature(T)
#         simulations.append(sim)
#     return simulations

# simulations = create_simulations(temperatures)

# # -------------------------
# # 交換試行関数：Metropolis判定
# # -------------------------
# def attempt_exchange(sim1, sim2, T1, T2):
#     state1 = sim1.context.getState(getEnergy=True, getPositions=True, getVelocities=True)
#     E1 = state1.getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
#     pos1 = state1.getPositions()
#     vel1 = state1.getVelocities()
    
#     state2 = sim2.context.getState(getEnergy=True, getPositions=True, getVelocities=True)
#     E2 = state2.getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
#     pos2 = state2.getPositions()
#     vel2 = state2.getVelocities()
    
#     kB = unit.MOLAR_GAS_CONSTANT_R.value_in_unit(unit.kilojoule_per_mole/unit.kelvin)
#     beta1 = 1.0/(kB*T1.value_in_unit(unit.kelvin))
#     beta2 = 1.0/(kB*T2.value_in_unit(unit.kelvin))
    
#     delta = (beta1 - beta2)*(E2 - E1)
#     if delta < 0.0 or random.random() < math.exp(-delta):
#         # 交換成立
#         sim1.context.setPositions(pos2)
#         sim1.context.setVelocities(vel2)
#         sim2.context.setPositions(pos1)
#         sim2.context.setVelocities(vel1)
#         return True
#     return False

# # -------------------------
# # 温度調整用関数
# # -------------------------
# def adjust_temperatures(temperatures, exchange_attempts, exchange_success, target=0.25):
#     # temperatures: 現在の温度リスト
#     # exchange_attempts[i], exchange_success[i]: レプリカ i と i+1 の交換試行数と成功数
#     # target: 目標成功率(0.25)
    
#     # 現在の温度の逆数 (beta) リストを用意 (等間隔調整などの参考)
#     betas = [1.0/(unit.MOLAR_GAS_CONSTANT_R.value_in_unit(unit.kilojoule_per_mole/unit.kelvin)*T.value_in_unit(unit.kelvin)) for T in temperatures]
    
#     # 各ペアの交換成功率を計算
#     acceptance = []
#     for i in range(len(temperatures)-1):
#         if exchange_attempts[i] > 0:
#             acc = exchange_success[i]/exchange_attempts[i]
#         else:
#             acc = 0.0
#         acceptance.append(acc)
    
#     # 隣接温度間の調整
#     # 非常に単純な方針：成功率が高いペア -> 温度差を小さく
#     # 成功率が低いペア -> 温度差を大きく
#     # 以下はbetasを微調整し、それに基づいてtemperaturesを更新する例
#     #
#     # delta_beta[i] = betas[i+1]-betas[i] がペアiの温度差を支配
#     # 目標交換率からのズレによって delta_beta をスケール

#     for i, acc in enumerate(acceptance):
#         # 誤差
#         diff = acc - target
#         # diff > 0 (成功率が高い)：温度差を小さくしたい => beta差を小さくする
#         # diff < 0 (成功率が低い)：温度差を大きくしたい => beta差を大きくする
#         # 下では調整強度を0.1程度とする（適宜調整が必要）
#         adjust_factor = 1.0 - 0.1*diff  # acc=0.25なら1.0に近い値
#         # betas[i+1]とbetas[i]の中点を固定して差分を調整する簡易法
#         mid_beta = 0.5*(betas[i] + betas[i+1])
#         half_diff = 0.5*(betas[i+1]-betas[i])*adjust_factor
#         betas[i] = mid_beta - half_diff
#         betas[i+1] = mid_beta + half_diff
    
#     # betasから温度を再計算
#     new_temperatures = []
#     for b in betas:
#         new_T = 1.0/(b*unit.MOLAR_GAS_CONSTANT_R.value_in_unit(unit.kilojoule_per_mole/unit.kelvin))
#         new_temperatures.append(new_T*unit.kelvin)
    
#     return new_temperatures

# # -------------------------
# # メインループ: ステップ実行と交換試行 + 温度調整
# # -------------------------
# steps_per_exchange = 1000
# n_iterations = 100
# n_steps_adjust = 10  # 10回交換後に温度調整
# target_acceptance = 0.25

# exchange_attempts = [0]*(n_replicas-1)
# exchange_success = [0]*(n_replicas-1)

# for iteration in range(n_iterations):
#     # 各レプリカを同数のステップ進める
#     for sim in simulations:
#         sim.step(steps_per_exchange)
    
#     # 隣接レプリカ間で交換試行
#     for i in range(n_replicas-1):
#         exchange_attempts[i] += 1
#         success = attempt_exchange(simulations[i], simulations[i+1], 
#                                    temperatures[i], temperatures[i+1])
#         if success:
#             exchange_success[i] += 1
    
#     # 一定間隔で温度分布を調整
#     if (iteration+1) % n_steps_adjust == 0 and iteration > 0:
#         # 現在の交換成功率を踏まえ温度を更新
#         temperatures = adjust_temperatures(temperatures, exchange_attempts, exchange_success, target=target_acceptance)
        
#         # 新しい温度に合わせてSimulationを作り直し(必要なら）
#         # 通常はpositions/velocitiesを保持しつつContextのみ再初期化する方が望ましいが、ここでは簡易化
#         # 既存シミュレーションのContextに対してIntegratorだけ変える、あるいは温度スケールし直すなど
#         # 簡易法として、positions/velocitiesを保持してintegratorだけ変えなおす例：
#         positions = [sim.context.getState(getPositions=True).getPositions() for sim in simulations]
#         velocities = [sim.context.getState(getVelocities=True).getVelocities() for sim in simulations]
        
#         simulations = []
#         for i, T in enumerate(temperatures):
#             integrator = mm.LangevinIntegrator(T, 1.0/unit.picosecond, 0.002*unit.picoseconds)
#             sim = app.Simulation(pdb.topology, system, integrator)
#             sim.context.setPositions(positions[i])
#             sim.context.setVelocities(velocities[i])
#             simulations.append(sim)
        
#         # 統計をリセットまたは継続するか判断
#         # ここでは継続
#         # (もし統計をリセットするなら exchange_attempts = [0]*(n_replicas-1), exchange_success = [0]*(n_replicas-1)など)
        
#         print("Iteration:", iteration+1, "Adjusted temperatures:", [T.value_in_unit(unit.kelvin) for T in temperatures])
#         print("Current acceptance rates:", [exchange_success[j]/exchange_attempts[j] if exchange_attempts[j] > 0 else 0 for j in range(n_replicas-1)])


# # 最終的には temperatures の分布が目標交換率付近に収束することが期待される


## 最終構造を保存

In [16]:
# シミュレーション結果(PDB)を保存
for i, sim in enumerate(simulations):
    state = sim.context.getState(getPositions=True, getEnergy=True)
    with open(f"{output_dir}/output_replica_{i}.pdb", "w") as f:
        app.PDBFile.writeFile(pdb.topology, state.getPositions(), f)

In [17]:
# パラメータをJSONに保存
import json
filepath = f"{output_dir}/params.json"
with open(filepath, mode="wt", encoding="utf-8") as f:
	json.dump(params, f, ensure_ascii=False, indent=2)

In [18]:
print(params['temperatures'])

[300.0, 318.2907697624934, 341.75128605048053, 370.56976918325563, 402.1110940275931, 434.4562284083984, 464.0497844861529, 495.2409261648359]
